In [ ]:
import pandas as pd
from pandas_datareader import data as wb
from yahoo_fin.stock_info import *
import requests
import plotly.graph_objects as go
import yfinance as yf
import numpy as np
import numpy_financial as npf
import lxml
from lxml import html
from bs4 import BeautifulSoup as bs
BS = []
IS = []
cfs = []
years = []
summary = []

In [ ]:
def get_data(stock):
    global BS
    global IS
    global cfs
    global years
    global summary
    IS=get_income_statement(stock)
    BS=get_balance_sheet(stock)
    cfs=get_cash_flow(stock)
    years=BS.columns
    summary=get_quote_table(stock)


In [ ]:
def DCF(stock):
    get_data(stock)
    risk_free=0.0169
    risk_premium=0.056
    g=0.02
    beta=summary['Beta (5Y Monthly)']
    cost_of_equity=risk_free+risk_premium*beta
     
    
    revenue_g=(IS[years[0]]['totalRevenue']-IS[years[1]]['totalRevenue'])/IS[years[1]]['totalRevenue']
    income_statement = pd.DataFrame.from_dict(IS[years[0]])
    income_statement.columns=['current_year']
    
    if income_statement['current_year']['interestExpense']==None:
        income_statement['current_year']['interestExpense']=int(input('Enter the Interest Expense of the latest year of ' + stock + ' '))
        income_statement['current_year']['interestExpense']=income_statement['current_year']['interestExpense']*-1
        
    income_statement['as_%_of_revenue']=income_statement/income_statement['current_year']['totalRevenue']
        
    #forecasting the next 5 years in the income statement
    income_statement['next_year'] =  (income_statement['current_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
    income_statement['next_2_year'] =  (income_statement['next_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
    income_statement['next_3_year'] =  (income_statement['next_2_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
    income_statement['next_4_year'] =  (income_statement['next_3_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue'] 
    income_statement['next_5_year'] =  (income_statement['next_4_year']['totalRevenue'] * (1+revenue_g)) * income_statement['as_%_of_revenue']
    
    balance_sheet = pd.DataFrame.from_dict(BS[years[0]])
    balance_sheet.columns=['current_year']
    balance_sheet['as_%_of_revenue']=balance_sheet/income_statement['current_year']['totalRevenue']
    
    #forecasting the next 5 years in the Balance Sheet.
    balance_sheet['next_year'] =  income_statement['next_year']['totalRevenue'] * balance_sheet['as_%_of_revenue']
    balance_sheet['next_2_year'] =  income_statement['next_2_year'] ['totalRevenue'] * balance_sheet['as_%_of_revenue']
    balance_sheet['next_3_year'] =  income_statement['next_3_year']['totalRevenue'] * balance_sheet['as_%_of_revenue'] 
    balance_sheet['next_4_year'] =  income_statement['next_4_year']['totalRevenue']  * balance_sheet['as_%_of_revenue'] 
    balance_sheet['next_5_year'] =  income_statement['next_5_year']['totalRevenue'] * balance_sheet['as_%_of_revenue']
        
    cash_flow = pd.DataFrame.from_dict(cfs[years[0]])
    cash_flow.columns=['current_year']
    
    #forecasting depreciation
    depreciation_per = cash_flow['current_year']['depreciation']/income_statement['current_year']['totalRevenue']
    depreciation_fut = {}
    depreciation_fut['next_year'] = income_statement['next_year']['totalRevenue']*depreciation_per
    depreciation_fut['next_2_year'] = income_statement['next_2_year']['totalRevenue']*depreciation_per
    depreciation_fut['next_3_year'] = income_statement['next_3_year']['totalRevenue']*depreciation_per
    depreciation_fut['next_4_year'] = income_statement['next_4_year']['totalRevenue']*depreciation_per
    depreciation_fut['next_5_year'] = income_statement['next_5_year']['totalRevenue']*depreciation_per
    
    #forecasting the cash flow
    nopat={}
    gross_cash_flow={}
    inc_receivables={}
    inc_inventory={}
    inc_payables={}
    inv_working_capital={}
    capex={}
    other_assets_liabilities={}
    free_cash_flow_forecast={}
    
    #current year
    nopat['current_year']=income_statement['current_year']['ebit']-income_statement['current_year']['incomeTaxExpense']
    gross_cash_flow['current_year']=nopat['current_year']+cash_flow['current_year']['depreciation']
    
    inc_receivables['current_year']=BS[years[0]]['netReceivables'] - BS[years[1]]['netReceivables']
    try:
        inc_inventory['current_year']=BS[years[0]]['inventory'] - BS[years[1]]['inventory']
    except:
        pass
    inc_payables['current_year']=BS[years[0]]['accountsPayable'] - BS[years[1]]['accountsPayable']  
    
    try:
        inv_working_capital['current_year']=(inc_receivables['current_year'] *-1)+(inc_inventory['current_year']*-1)+inc_payables['current_year']
    except:
        inv_working_capital['current_year']=(inc_receivables['current_year'] *-1)+inc_payables['current_year']
        pass
    capex['current_year']=BS[years[0]]['propertyPlantEquipment']-BS[years[1]]['propertyPlantEquipment']+cash_flow['current_year']['depreciation']
    
    free_cash_flow_low = gross_cash_flow['current_year'] + inv_working_capital['current_year'] - capex['current_year']
    real_current_free_cash=cash_flow['current_year']['totalCashFromOperatingActivities']+cash_flow['current_year']['capitalExpenditures']
    other_assets_liabilities['current_year'] = real_current_free_cash - free_cash_flow_low
    
    other_per_revenue = other_assets_liabilities['current_year'] / income_statement['current_year']['totalRevenue']
    
        
    #next year
    nopat['next_year']=income_statement['next_year']['ebit']-income_statement['next_year']['incomeTaxExpense']
    gross_cash_flow['next_year']=nopat['next_year']+depreciation_fut['next_year']
    
    inc_receivables['next_year']=balance_sheet['next_year']['netReceivables'] - balance_sheet['current_year']['netReceivables']
    try:
        inc_inventory['next_year']=balance_sheet['next_year']['inventory'] - balance_sheet['current_year']['inventory']
    except:
        pass
    inc_payables['next_year']=balance_sheet['next_year']['accountsPayable'] - balance_sheet['current_year']['accountsPayable']
    
    try:
        inv_working_capital['next_year']=(inc_receivables['next_year'] *-1)+(inc_inventory['next_year']*-1)+inc_payables['next_year']
    except:
        inv_working_capital['next_year']=(inc_receivables['next_year'] *-1)+inc_payables['next_year']
        pass
    capex['next_year']=balance_sheet['next_year']['propertyPlantEquipment']-balance_sheet['current_year']['propertyPlantEquipment']+depreciation_fut['next_year']
    other_assets_liabilities['next_year'] = income_statement['next_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_year']= gross_cash_flow['next_year'] + inv_working_capital['next_year'] - capex['next_year'] + other_assets_liabilities['next_year']
    
    #next 2 year
    nopat['next_2_year']=income_statement['next_2_year']['ebit']-income_statement['next_2_year']['incomeTaxExpense']
    gross_cash_flow['next_2_year']=nopat['next_2_year']+depreciation_fut['next_2_year']
    
    inc_receivables['next_2_year']=balance_sheet['next_2_year']['netReceivables'] - balance_sheet['next_year']['netReceivables']
    try:
        inc_inventory['next_2_year']=balance_sheet['next_2_year']['inventory'] - balance_sheet['next_year']['inventory']
    except:
        pass
    inc_payables['next_2_year']=balance_sheet['next_2_year']['accountsPayable'] - balance_sheet['next_year']['accountsPayable']
    
    try:
        inv_working_capital['next_2_year']=(inc_receivables['next_2_year'] *-1)+(inc_inventory['next_2_year']*-1)+inc_payables['next_2_year']
    except:
        inv_working_capital['next_2_year']=(inc_receivables['next_2_year'] *-1)+inc_payables['next_2_year']
        pass
    capex['next_2_year']=balance_sheet['next_2_year']['propertyPlantEquipment']-balance_sheet['next_year']['propertyPlantEquipment']+depreciation_fut['next_2_year']
    other_assets_liabilities['next_2_year'] = income_statement['next_2_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_2_year']= gross_cash_flow['next_2_year'] + inv_working_capital['next_2_year'] - capex['next_2_year'] + other_assets_liabilities['next_2_year']
    
    #next 3 year
    nopat['next_3_year']=income_statement['next_3_year']['ebit']-income_statement['next_3_year']['incomeTaxExpense']
    gross_cash_flow['next_3_year']=nopat['next_3_year']+depreciation_fut['next_3_year']
    
    inc_receivables['next_3_year']=balance_sheet['next_3_year']['netReceivables'] - balance_sheet['next_2_year']['netReceivables']
    try:
        inc_inventory['next_3_year']=balance_sheet['next_3_year']['inventory'] - balance_sheet['next_2_year']['inventory']
    except:
        pass
    inc_payables['next_3_year']=balance_sheet['next_3_year']['accountsPayable'] - balance_sheet['next_2_year']['accountsPayable']
    
    try:
        inv_working_capital['next_3_year']=(inc_receivables['next_3_year'] *-1)+(inc_inventory['next_3_year']*-1)+inc_payables['next_3_year']
    except:
        inv_working_capital['next_3_year']=(inc_receivables['next_3_year'] *-1)+inc_payables['next_3_year']
        pass
    capex['next_3_year']=balance_sheet['next_3_year']['propertyPlantEquipment']-balance_sheet['next_2_year']['propertyPlantEquipment']+depreciation_fut['next_3_year']
    other_assets_liabilities['next_3_year'] = income_statement['next_3_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_3_year']= gross_cash_flow['next_3_year'] + inv_working_capital['next_3_year'] - capex['next_3_year'] + other_assets_liabilities['next_3_year']
    
    #next 4 year
    nopat['next_4_year']=income_statement['next_4_year']['ebit']-income_statement['next_4_year']['incomeTaxExpense']
    gross_cash_flow['next_4_year']=nopat['next_4_year']+depreciation_fut['next_4_year']
    
    inc_receivables['next_4_year']=balance_sheet['next_4_year']['netReceivables'] - balance_sheet['next_3_year']['netReceivables']
    try:
        inc_inventory['next_4_year']=balance_sheet['next_4_year']['inventory'] - balance_sheet['next_3_year']['inventory']
    except:
        pass
    inc_payables['next_4_year']=balance_sheet['next_4_year']['accountsPayable'] - balance_sheet['next_3_year']['accountsPayable']
    
    try:
        inv_working_capital['next_4_year']=(inc_receivables['next_4_year'] *-1)+(inc_inventory['next_4_year']*-1)+inc_payables['next_4_year']
    except:
        inv_working_capital['next_4_year']=(inc_receivables['next_4_year'] *-1)+inc_payables['next_4_year']
        pass
    capex['next_4_year']=balance_sheet['next_4_year']['propertyPlantEquipment']-balance_sheet['next_4_year']['propertyPlantEquipment']+depreciation_fut['next_4_year']
    other_assets_liabilities['next_4_year'] = income_statement['next_4_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_4_year']= gross_cash_flow['next_4_year'] + inv_working_capital['next_4_year'] - capex['next_4_year'] + other_assets_liabilities['next_4_year']
    
    #next 5 year
    nopat['next_5_year']=income_statement['next_5_year']['ebit']-income_statement['next_5_year']['incomeTaxExpense']
    gross_cash_flow['next_5_year']=nopat['next_5_year']+depreciation_fut['next_5_year']
    
    inc_receivables['next_5_year']=balance_sheet['next_5_year']['netReceivables'] - balance_sheet['next_4_year']['netReceivables']
    try:
        inc_inventory['next_5_year']=balance_sheet['next_5_year']['inventory'] - balance_sheet['next_4_year']['inventory']
    except:
        pass
    inc_payables['next_5_year']=balance_sheet['next_5_year']['accountsPayable'] - balance_sheet['next_4_year']['accountsPayable']
    
    try:
        inv_working_capital['next_5_year']=(inc_receivables['next_5_year'] *-1)+(inc_inventory['next_5_year']*-1)+inc_payables['next_5_year']
    except:
        inv_working_capital['next_5_year']=(inc_receivables['next_5_year'] *-1)+inc_payables['next_5_year']
        pass
    capex['next_5_year']=balance_sheet['next_5_year']['propertyPlantEquipment']-balance_sheet['next_5_year']['propertyPlantEquipment']+depreciation_fut['next_5_year']
    other_assets_liabilities['next_5_year'] = income_statement['next_5_year']['totalRevenue']*other_per_revenue
    free_cash_flow_forecast['next_5_year']= gross_cash_flow['next_5_year'] + inv_working_capital['next_5_year'] - capex['next_5_year'] + other_assets_liabilities['next_5_year']
    
        
    #getting the equity as a market cap
    equity=wb.get_quote_yahoo('AAPL')['marketCap']
    
    #scraping the total debt from yahoo finance
    balance_sheet_url = (f'https://finance.yahoo.com/quote/{stock}/balance-sheet?p={stock}')
    balance_sheet_html = requests.get(balance_sheet_url)
    balance_sheet_soup = bs(balance_sheet_html.text, 'html.parser')
    balance_sheet_table = balance_sheet_soup.find('div', class_='D(tbrg)')

    total_debt_lst = []
    total_debt_row = balance_sheet_table.find('div', attrs={'title':'Total Debt'}).parent.parent
    for value in total_debt_row.find_all('div'):
        value = value.text
        value = value.replace(',','')
        total_debt_lst.append(value)
    total_debt_int = int(total_debt_lst[3])
    total_debt_int=total_debt_int*1000
        
    
    #calculating the variables needed to calculate wacc    
    equity1=equity/(total_debt_int+equity)
    debt1=total_debt_int/(total_debt_int+equity)
    cost_of_debt=(income_statement['current_year']['interestExpense']*-1)/total_debt_int        
    wacc=debt1*cost_of_debt*(1-0.21)+equity1*cost_of_equity
    
    
    wacc_stock=wacc[0]
    values=free_cash_flow_forecast.values()
    fcf_list=list(values)
    #calculating the net present value of the sum of all the free cash flows forecasted
    npv=npf.npv(wacc_stock,fcf_list)
    
    
    #calculating the terminal value, i.e. the value of the company after 5 years
    Terminal_value=(free_cash_flow_forecast['next_5_year']*(1+g))/(wacc_stock-g)
    Terminal_value_Discounted = Terminal_value/(1+wacc_stock)**5
    Terminal_value_Discounted
    
    enterprise_value=npv+Terminal_value_Discounted
    cash=balance_sheet['current_year']['cash']
    equity_value=enterprise_value+cash-total_debt_int
    
    shares=wb.get_quote_yahoo(stock)['sharesOutstanding']
    forecasted_price=equity_value/shares
    
    forecasted_price_pure=forecasted_price[0]
    equity_value_mln=equity_value/1000000
    real_price=wb.get_quote_yahoo(stock)['price']
    real_price_pure=real_price[0]
    
    print(stock + ' Current price per stock is ' + str(real_price_pure) )
    print(stock + ' forecasted price per stock is ' + str(forecasted_price_pure) )
    print('Equity value of '+ stock + " is " + str(equity_value_mln) + " mln" )
    print('the forecast is based on the following assumptions: '+ 'revenue growth: ' + str(revenue_g) + ' Cost of Capital: ' + str(wacc_stock) )
    print('perpetuity growth: ' + str(g))
    

In [ ]:
while True:
    command=input('Write a stock ticker ')
    
    try:
        stock=command.split(' ')[1]
    except:
        pass
                
    if command =='DCF ' + stock:
        DCF(stock)
               
    elif command=='quit':
        break
        
    else:
        print('Statement is not valid')